In [1]:
from google.colab import files

# Upload kaggle.json file
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [3]:
!kaggle datasets download -d ananthu017/emotion-detection-fer


 92% 60.0M/65.2M [00:00<00:00, 147MB/s] 
100% 65.2M/65.2M [00:00<00:00, 124MB/s]


In [4]:
!unzip emotion-detection-fer.zip


Streaming output truncated to the last 5000 lines.
  inflating: train/sad/im37.png      
  inflating: train/sad/im370.png     
  inflating: train/sad/im3700.png    
  inflating: train/sad/im3701.png    
  inflating: train/sad/im3702.png    
  inflating: train/sad/im3703.png    
  inflating: train/sad/im3704.png    
  inflating: train/sad/im3705.png    
  inflating: train/sad/im3706.png    
  inflating: train/sad/im3707.png    
  inflating: train/sad/im3708.png    
  inflating: train/sad/im3709.png    
  inflating: train/sad/im371.png     
  inflating: train/sad/im3710.png    
  inflating: train/sad/im3711.png    
  inflating: train/sad/im3712.png    
  inflating: train/sad/im3713.png    
  inflating: train/sad/im3714.png    
  inflating: train/sad/im3715.png    
  inflating: train/sad/im3716.png    
  inflating: train/sad/im3717.png    
  inflating: train/sad/im3718.png    
  inflating: train/sad/im3719.png    
  inflating: train/sad/im372.png     
  inflating: train/sad/im3720.png    

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.optimizers import Adam

In [6]:
train_dir = 'train'
test_dir = 'test'

In [7]:
# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Normalization for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(48, 48),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [9]:
base_model = DenseNet121(weights='imagenet', include_top=False)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))

29084464/29084464 [==============================] - 0s 0us/step


In [10]:
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(
        train_generator,
        steps_per_epoch=int(train_generator.samples / train_generator.batch_size),
        epochs=20,
        validation_data=test_generator,
        validation_steps=int(test_generator.samples / test_generator.batch_size),
        verbose=1)


Epoch 1/20
897/897 [==============================] - 84s 94ms/step - loss: 1.7480 - accuracy: 0.3654 - val_loss: 1.5214 - val_accuracy: 0.4551
Epoch 2/20
897/897 [==============================] - 75s 84ms/step - loss: 1.4778 - accuracy: 0.4533 - val_loss: 1.3786 - val_accuracy: 0.4967
Epoch 3/20
897/897 [==============================] - 73s 82ms/step - loss: 1.3464 - accuracy: 0.4938 - val_loss: 1.2577 - val_accuracy: 0.5258
Epoch 4/20
897/897 [==============================] - 73s 82ms/step - loss: 1.2358 - accuracy: 0.5370 - val_loss: 1.2278 - val_accuracy: 0.5453
Epoch 5/20
897/897 [==============================] - 73s 82ms/step - loss: 1.1488 - accuracy: 0.5702 - val_loss: 1.1925 - val_accuracy: 0.5703
Epoch 6/20
897/897 [==============================] - 72s 81ms/step - loss: 1.0580 - accuracy: 0.6130 - val_loss: 1.1427 - val_accuracy: 0.5896
Epoch 7/20
897/897 [==============================] - 72s 81ms/step - loss: 0.9532 - accuracy: 0.6527 - val_loss: 1.1423 - val_accuracy:

In [14]:
model.save("datathon5_V1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
import numpy as np
from sklearn.metrics import classification_report

predictions = model.predict(test_generator)

predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

report = classification_report(true_classes, predicted_classes, target_names=class_labels)

print(report)

225/225 [==============================] - 8s 29ms/step
              precision    recall  f1-score   support

       angry       0.51      0.57      0.54       958
   disgusted       0.76      0.60      0.67       111
     fearful       0.48      0.50      0.49      1024
       happy       0.77      0.84      0.80      1774
     neutral       0.55      0.57      0.56      1233
         sad       0.52      0.40      0.46      1247
   surprised       0.78      0.74      0.76       831

    accuracy                           0.62      7178
   macro avg       0.63      0.60      0.61      7178
weighted avg       0.61      0.62      0.61      7178



In [33]:
import numpy as np
from PIL import Image

def predict_emotion(image_path, model):

    img = Image.open(image_path)
    img = img.resize((48, 48))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    img_array = np.expand_dims(img_array, axis=-1)

    img_array = np.concatenate([img_array, img_array, img_array], axis=-1)

    predictions = model.predict(img_array)


    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    predicted_class_index = np.argmax(predictions)
    predicted_emotion = emotion_labels[predicted_class_index]

    return predicted_emotion

image_path = 'im15.png'
predicted_emotion = predict_emotion(image_path, model)
print("Predicted Emotion:", predicted_emotion)

1/1 [==============================] - 0s 48ms/step
Predicted Emotion: Angry
